In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
from my_functions import *

In [ ]:
tcurves = load_tcurves(load_filter_tags())
w_central = central_wavelength(tcurves)

In [ ]:
## Load the catalog
with open('pkl/cat_flambda_photoz_gaia.pkl', 'rb') as file:
    cat = pkl.load(file)

tcurves = load_tcurves(load_filter_tags())
filters_tags = load_filter_tags()

pm_flx = cat['flx'].T * 1e-19
pm_err = cat['flx_err'].T * pm_flx

In [ ]:
# Function to drop sources
def make_masks(cat, nb_ind):
    # Mask sources with high photoz odds
    mask_pz_odds = cat['odds'] < 0.98
    # Mask sources with proper motion according to Gaia (sigma > 3)
    parallax_sn = np.abs(cat['parallax'] / cat['parallax_err'])
    pmra_sn = np.abs(cat['pmra'] / cat['pmra_err'])
    pmdec_sn = np.abs(cat['pmdec'] / cat['pmdec_err'])
    mask_pmotion = (
        (np.sqrt(parallax_sn**2 + pmra_sn**2 + pmdec_sn**2) < 27.**0.5)
        | ( np.isnan(parallax_sn) | np.isnan(pmra_sn) | np.isnan(pmdec_sn) )
    )
    # Mask sources with SNR < 5 in the selected NB
    mask_snr = cat['flx_err'][:,nb_ind] < 0.2
    # Mask too bright gSDSS
    mask_b = cat['flx'][:, -3] > mag_to_flux(15, 4750)
    
    mask_total = mask_pmotion & mask_snr & mask_b # & mask_pz_odds
    # print('Total masked: {}'.format(len(np.where(~mask_total)[0])))
    return mask_total

In [ ]:
N_nb = 6 # Number of nb on each side of the central one
ew0min = 30

N_sources = len(cat['flx'])

In [ ]:
nb_c_min = 6
nb_c_max = 54
line = np.zeros((nb_c_max - nb_c_min, N_sources)).astype(bool)
cont_est_Arr = np.zeros((nb_c_max - nb_c_min, N_sources))
cont_err_Arr = np.zeros((nb_c_max - nb_c_min, N_sources))
i = 0
for nb_c in range(nb_c_min, nb_c_max):
    z_nb = w_central[nb_c] / 1215.67 - 1
    fwhm = nb_fwhm(nb_c, tcurves=tcurves)
    cont_est, cont_err = stack_estimation(pm_flx, pm_err, nb_c, N_nb, False)
    line[i] = (
        (pm_flx[nb_c] - cont_est > 3 * (cont_err**2 + pm_err[nb_c]**2) ** 0.5)
        & (pm_flx[nb_c] - cont_est > ew0min * (1 + z_nb) * cont_est / fwhm)
    )
    cont_est_Arr[i] = cont_est
    cont_err_Arr[i] = cont_err
    i += 1

In [ ]:
# Arr where the center of the Lya line is. -1 means no lines detected
line_lya_center = np.ones(N_sources, dtype=int) * -1

for src_n in range(N_sources):
    try:
        first = np.where(line[:, src_n])[0][0] # First True in line_qso
    except:
        continue
    if first > 40: # We don't expect lines past here
        continue
    if ~make_masks(cat, first)[src_n]: continue

    line_nb_width = 0
    while line[first + line_nb_width + 1, src_n]:
        line_nb_width += 1

    line_lya_center[src_n] = np.argmax(
        pm_flx[first + nb_c_min : first + nb_c_min + line_nb_width + 1, src_n]
    ) + first + nb_c_min

print(len(np.where((line_lya_center != -1))[0]))

In [ ]:
line_CIV = np.zeros(N_sources, dtype=bool)
nb_CIV_Arr = np.ones(N_sources, dtype=int) * -1
for src_n in np.where(line_lya_center != -1)[0]:
    w_obs_lya = w_central[line_lya_center[src_n]]
    z = w_obs_lya / 1215.67 - 1
    w_obs_CIV = 1549.48 * (1 + z)

    if w_obs_CIV > 8500.: continue # Out of range

    nb_CIV_Arr[src_n] = np.argmin(np.abs(w_central[:-4] - w_obs_CIV))
    nb_CIV = nb_CIV_Arr[src_n]
    line_CIV[src_n] = pm_flx[nb_CIV, src_n] - cont_est_Arr[nb_CIV - nb_c_min, src_n]\
        > 3 * (cont_err_Arr[nb_CIV - nb_c_min, src_n]**2 + pm_err[nb_CIV, src_n]**2)\
            ** 0.5
print(len(np.where(line_CIV)[0]))

In [ ]:
## CIV line
j = 0
for src_n in np.where(line_CIV)[0]:
    #if not make_masks(cat, nb_CIV_Arr[src_n])[src_n]: continue # Masked source

    fig = plt.figure(figsize=(10, 8))
    ax = plot_JPAS_source(pm_flx[:, src_n], pm_err[:, src_n])

    ax.errorbar(w_central[line_lya_center[src_n]],
        cont_est_Arr[line_lya_center[src_n], src_n],
        yerr=cont_err_Arr[line_lya_center[src_n], src_n],
        fmt='^', c='C4', markersize=10, capsize=3, capthick=3, elinewidth=4)

    ax.errorbar(w_central[nb_CIV_Arr[src_n]],
        cont_est_Arr[nb_CIV_Arr[src_n], src_n],
        yerr=cont_err_Arr[nb_CIV_Arr[src_n], src_n],
        fmt='^', c='C6', markersize=10, capsize=3, capthick=3, elinewidth=4)

    print('z NB   = {0:0.2f}'.format(w_central[line_lya_center[src_n]] / 1215.67 - 1))

    plt.show()

    j += 1
    if j == 10: break

In [ ]:
line_CIII = np.zeros(N_sources, dtype=bool)
nb_CIII_Arr = np.ones(N_sources, dtype=int) * -1
for src_n in np.where(line_CIV)[0]:
    w_obs_lya = w_central[line_lya_center[src_n]]
    z = w_obs_lya / 1215.67 - 1
    w_obs_CIII = 1908.73 * (1 + z)

    if w_obs_CIII > 8200.: continue # Out of range

    nb_CIII_Arr[src_n] = np.argmin(np.abs(w_central[:-4] - w_obs_CIII))
    nb_CIII = nb_CIII_Arr[src_n]
    line_CIII[src_n] = pm_flx[nb_CIII, src_n] - cont_est_Arr[nb_CIII - nb_c_min, src_n]\
        > 3 * (cont_err_Arr[nb_CIII - nb_c_min, src_n]**2 + pm_err[nb_CIII, src_n]**2)\
            ** 0.5
print(len(np.where(line_CIII)[0]))

In [ ]:
## CIII] line
j = 0
for src_n in np.where(line_CIII)[0]:
    fig = plt.figure(figsize=(10, 8))
    ax = plot_JPAS_source(pm_flx[:, src_n], pm_err[:, src_n])

    ax.errorbar(w_central[line_lya_center[src_n]],
        cont_est_Arr[line_lya_center[src_n], src_n],
        yerr=cont_err_Arr[line_lya_center[src_n], src_n],
        fmt='^', c='C4', markersize=10, capsize=3, capthick=3, elinewidth=4)

    ax.errorbar(w_central[nb_CIV_Arr[src_n]],
        cont_est_Arr[nb_CIV_Arr[src_n], src_n],
        yerr=cont_err_Arr[nb_CIV_Arr[src_n], src_n],
        fmt='^', c='C6', markersize=10, capsize=3, capthick=3, elinewidth=4)

    ax.errorbar(w_central[nb_CIII_Arr[src_n]],
        cont_est_Arr[nb_CIII_Arr[src_n], src_n],
        yerr=cont_err_Arr[nb_CIII_Arr[src_n], src_n],
        fmt='^', c='C7', markersize=10, capsize=3, capthick=3, elinewidth=4)

    plt.show()

    j += 1
    if j == 10: break